In [3]:
import pandas as pd

In [4]:
train_data = pd.read_csv("Train.csv")
test_data = pd.read_csv("Test.csv")

In [5]:
train_data

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...
10975,569934458364813313,neutral,American,NaN,Cottopanama85,NaN,0,@AmericanAir followback,NaN,2015-02-23 10:58:58 -0800,"ohio,panama",NaN
10976,568564006329434113,positive,United,NaN,PaulBEsteves,NaN,0,@united thanks for the help. Wish the phone re...,NaN,2015-02-19 16:13:17 -0800,Brooklyn,Eastern Time (US & Canada)
10977,569643648910028801,negative,US Airways,NaN,runfixsteve,NaN,0,@usairways the. Worst. Ever. #dca #customerser...,NaN,2015-02-22 15:43:24 -0800,"St. Augustine, Florida",NaN
10978,568864981917110272,negative,US Airways,NaN,CLChicosky,NaN,0,@nrhodes85: look! Another apology. DO NOT FLY ...,NaN,2015-02-20 12:09:15 -0800,NaN,NaN


In [6]:
train_texts = train_data["text"]
train_sentiments = train_data["airline_sentiment"]

test_texts = test_data["text"]

In [7]:
def sentimentToLabel(sentiment):
    if (sentiment == "positive"):
        return 1
    elif (sentiment == "negative"):
        return -1
    return 0

def labelToSentiment(label):
    if (label == 1):
        return "positive"
    elif (label == -1):
        return "negative"
    return "neutral"

In [8]:
from nltk import TweetTokenizer, pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import string

import nltk
nltk.download("stopwords")
nltk.download("averaged_perceptron_tagger")
nltk.download("wordnet")

stops = stopwords.words("english") + list(string.punctuation)

tokenizer = TweetTokenizer()
lemmatizer = WordNetLemmatizer()

def get_small_pos(word):
    pos = pos_tag([word])[0][1]
    if pos.startswith('J'):
        return wordnet.ADJ
    elif pos.startswith('V'):
        return wordnet.VERB
    elif pos.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def cleanText(text):
    tokens = tokenizer.tokenize(text)
    # No need of Removing stop words here, will be removed using max_df
    words = [lemmatizer.lemmatize(w, get_small_pos(w)) for w in tokens if w.lower() not in stops or w.lower() == "not"]
    return " ".join(words)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/smriti/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/smriti/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/smriti/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
# Training Data
Y_train = train_sentiments.apply(sentimentToLabel)
train_cleaned_texts = train_texts.apply(cleanText)

# Testing Data
test_cleaned_texts = test_texts.apply(cleanText)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# cv = CountVectorizer(max_features = 10000, max_df=0.25, tokenizer = tokenizer.tokenize, ngram_range=(1, 2))
cv = TfidfVectorizer(max_features = 3000, tokenizer = tokenizer.tokenize, ngram_range=(1, 2))

X_train = cv.fit_transform(train_cleaned_texts).todense()
X_test = cv.transform(test_cleaned_texts).todense()

In [11]:
cv.get_feature_names()

['#americanairlines',
 '#avgeek',
 '#badcustomerservice',
 '#badservice',
 '#cancelled',
 '#customerservice',
 '#dca',
 '#delayed',
 '#destinationdragons',
 '#disappointed',
 '#epicfail',
 '#fail',
 '#flight',
 '#flights',
 '#flyfi',
 '#flyingitforward',
 '#frustrated',
 '#help',
 '#jetblue',
 '#jfk',
 '#neveragain',
 '#notcool',
 '#nothappy',
 '#poorservice',
 '#ridiculous',
 '#ripoff',
 '#southwest',
 '#travel',
 '#unacceptable',
 '#unfriendlyskies',
 '#united',
 '#unitedairlines',
 '#unitedfail',
 '#usairways',
 '#usairwaysfail',
 '#wtf',
 '->',
 '..',
 '...',
 '... get',
 '... need',
 '... not',
 '... please',
 '... still',
 '... thanks',
 '0',
 '1',
 '1 hour',
 '1 hr',
 '1.5',
 '1.5 hour',
 '1.5 hr',
 '1/2',
 '1/2 hour',
 '10',
 '10 hour',
 '10 hr',
 '10 min',
 '10 minute',
 '100',
 '1000',
 '10:30',
 '11',
 '11:30',
 '12',
 '12 hour',
 '13',
 '130',
 '14',
 '140',
 '15',
 '15 min',
 '15 minute',
 '150',
 '16',
 '17',
 '18',
 '19',
 '1hr',
 '1k',
 '1st',
 '1st class',
 '1st flight

## SVM

In [12]:
from sklearn.decomposition import PCA
pca = PCA()

In [13]:
pca.fit(X_train)

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [14]:
pca.explained_variance_

array([8.62996765e-03, 7.83122936e-03, 7.01556589e-03, ...,
       9.31525820e-37, 7.07728315e-37, 6.97598590e-37])

In [15]:
k = 0
total_var = sum(pca.explained_variance_)
curr_var = 0
while curr_var / total_var < 0.50:
    k += 1
    curr_var += pca.explained_variance_[k]

In [16]:
k

372

In [17]:
final_pca = PCA(n_components = 3000, whiten = True)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [18]:
from sklearn.svm import SVC
svc = SVC(C = 1000)

In [19]:
svc.fit(X_train_pca, Y_train)

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [ ]:
Y_pred = svc.predict(X_test_pca)

In [59]:
pd.DataFrame(Y_pred)[0].apply(labelToSentiment).to_csv("Results.csv", header=None, index=None)